# Using a list of Schistosome parasite genes, find compounds that are active on these targets:

* Aim: Find novel therapeutic targets for the parasite Schistosoma mansoni.  
* Background: Schistosome parasites kill 250,000 people every year. Treatment of schistosomiasis relies on the drug praziquantel (only!).   
* But what other targets could be druggable? And are there any existing marketed drugs for these targets?  
* For further detail see comment at https://www.science.org/doi/10.1126/science.abe0710  
* and scientific paper: https://www.science.org/doi/10.1126/science.abb7699  

## Method 1: Use the ChEMBL python client to access the API

* Available to install from Python Package Index by typing:  
pip install chembl_webresource_client  
* See https://github.com/chembl/chembl_webresource_client    
  
* Pros: simple to use, tailored for ChEMBL API endpoints
* Cons: The keyword 'only' is limited to main fields, so it can't search within nested fields (e.g. in the molecule API, .only(['molecule_properties__alogp']) is equivalent to .only(['molecule_properties']).  

In [17]:
###############################
#This cell imports relevant python modules:
###############################
import pandas as pd #Use pandas python module to view and analyse data
from chembl_webresource_client.new_client import new_client #Import ChEMBL python client

## 1a: Find compounds that are active on the specified targets: 

In [20]:
###############################
#Set up the call to the ChEMBL 'activity' API:
###############################
activities = new_client.activity.filter(target_chembl_id__in=['CHEMBL5552','CHEMBL1075195']  ##Specify a list of example targets (#Serine/threonine-protein kinase 25 (STK25) & Serine-threonine kinases TAO2)
                                        , pchembl_value__gte=5                               ##Specify a minimum threshold of the pChEMBL activity value. Note that pCHEMBL = -log10(IC50, XC50, AC50, Ki, Kd, potency). Greater than or equal to 5 (10um) is a typical minimum rule of thumb for binding activity between a compound and a protein target. 
                                        , assay_type='B'                                     ##Only look for Binding Assays
                                       ).only(['target_chembl_id', 'target_pref_name', 'parent_molecule_chembl_id' ## Specify which fields (columns) to extract
                                                               ,'molecule_chembl_id','molecule_pref_name', 'pchembl_value'])
#Convert the list of results into a Pandas dataframe:
act_df = pd.DataFrame(activities)
act_df

(83, 7)


,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,target_chembl_id,target_pref_name,value
0,CHEMBL574738,AST-487,CHEMBL574738,5.92,CHEMBL5552,Serine/threonine-protein kinase 25,1200.0
1,CHEMBL522892,DOVITINIB,CHEMBL522892,5.92,CHEMBL5552,Serine/threonine-protein kinase 25,1200.0
2,CHEMBL607707,PELITINIB,CHEMBL607707,5.47,CHEMBL5552,Serine/threonine-protein kinase 25,3400.0
3,CHEMBL191003,JNJ-7706621,CHEMBL191003,6.44,CHEMBL5552,Serine/threonine-protein kinase 25,360.0
4,CHEMBL608533,MIDOSTAURIN,CHEMBL608533,5.75,CHEMBL5552,Serine/threonine-protein kinase 25,1800.0
...,...,...,...,...,...,...,...
78,CHEMBL388978,STAUROSPORINE,CHEMBL388978,8.57,CHEMBL5552,Serine/threonine-protein kinase 25,2.71
79,CHEMBL388978,STAUROSPORINE,CHEMBL388978,7.93,CHEMBL1075195,Serine/threonine-protein kinase TAO2,1.17
80,CHEMBL388978,STAUROSPORINE,CHEMBL388978,8.72,CHEMBL5552,Serine/threonine-protein kinase 25,1.9
81,CHEMBL4569508,None,CHEMBL4569508,5.57,CHEMBL1075195,Serine/threonine-protein kinase TAO2,2700.0


## 1b: Using the active compounds from the previous step, and call the 'molecule' API to find their molecular properties etc, so that the compound list can be prioritised

In [3]:
###############################
#First find the list of compounds that are within the act_df dataframe:
###############################
cmpd_chembl_ids = list(set(act_df['molecule_chembl_id']))
print("There are {} compounds initially identified as active on the known targets. e.g.{}...".format(len(cmpd_chembl_ids),cmpd_chembl_ids[0:2]))

There are 55 compounds initially identified as active on the known targets. e.g.['CHEMBL4568087', 'CHEMBL1721885']...


In [5]:
###############################
#Set up the call to the ChEMBL 'molecule' API:
###############################

#Select a few examples of the active compounds from above (i.e. a reduced list!) : 
cmpd_chembl_ids = ",".join(['CHEMBL296468', 'CHEMBL180022', 'CHEMBL475251','CHEMBL535']) # Amend the format of the text string so that it is suitable for the API call

molecules = new_client.molecule.filter(molecule_chembl_id__in=cmpd_chembl_ids  ##Select a few examples of the active compounds (i.e. a reduced list!)
                                       ).only([ 'molecule_chembl_id','pref_name', 'molecule_hierarchy', 'molecule_properties', 'max_phase']) ## Specify which fields (columns) to extract

#Convert the list of results into a Pandas dataframe:
mol_df = pd.DataFrame(molecules)
mol_df

,max_phase,molecule_chembl_id,molecule_hierarchy,molecule_properties,pref_name
0,4,CHEMBL535,"{'molecule_chembl_id': 'CHEMBL535', 'parent_ch...","{'alogp': '3.33', 'aromatic_rings': 2, 'cx_log...",SUNITINIB
1,1,CHEMBL296468,"{'molecule_chembl_id': 'CHEMBL296468', 'parent...","{'alogp': '3.66', 'aromatic_rings': 2, 'cx_log...",BMS-387032
2,4,CHEMBL180022,"{'molecule_chembl_id': 'CHEMBL180022', 'parent...","{'alogp': '5.93', 'aromatic_rings': 4, 'cx_log...",NERATINIB
3,2,CHEMBL475251,"{'molecule_chembl_id': 'CHEMBL475251', 'parent...","{'alogp': '3.63', 'aromatic_rings': 3, 'cx_log...",R-406


In [6]:
###########################
# Convert nested cells (ie those containing a dictionary) to individual columns in the dataframe so that is it easier to filter!
###########################
# Molecule hierarchy: 
mol_df['parent_molecule_chembl_id'] = mol_df['molecule_hierarchy'].apply(lambda x: x['parent_chembl_id'])

#Physicochemical properties (only report if cells are not null)
mol_df['cx_logd'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_logd'])
mol_df['cx_logp'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_logp'])
mol_df['cx_most_apka'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_most_apka'])
mol_df['cx_most_bpka'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_most_bpka'])
mol_df['alogp'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['alogp'])
mol_df['hba'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['hba'])
mol_df['hbd'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['hbd'])
mol_df['mw_freebase'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['mw_freebase']) #This is the mwt of the parent compound
mol_df['full_mwt'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['full_mwt']) #This is the mwt of the full compound including any salt
mol_df['num_ro5_violations'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['num_ro5_violations'])
mol_df['psa'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['psa'])
mol_df['heavy_atoms'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['heavy_atoms'])

mol_df

(4, 18)


,max_phase,molecule_chembl_id,molecule_hierarchy,molecule_properties,pref_name,parent_molecule_chembl_id,cx_logd,cx_logp,cx_most_apka,cx_most_bpka,alogp,hba,hbd,mw_freebase,full_mwt,num_ro5_violations,psa,heavy_atoms
0,4,CHEMBL535,"{'molecule_chembl_id': 'CHEMBL535', 'parent_ch...","{'alogp': '3.33', 'aromatic_rings': 2, 'cx_log...",SUNITINIB,CHEMBL535,1.28,2.93,11.46,9.04,3.33,3,3,398.48,398.48,0,77.23,29
1,1,CHEMBL296468,"{'molecule_chembl_id': 'CHEMBL296468', 'parent...","{'alogp': '3.66', 'aromatic_rings': 2, 'cx_log...",BMS-387032,CHEMBL296468,0.33,0.94,7.94,10.17,3.66,7,2,380.54,380.54,0,80.05,25
2,4,CHEMBL180022,"{'molecule_chembl_id': 'CHEMBL180022', 'parent...","{'alogp': '5.93', 'aromatic_rings': 4, 'cx_log...",NERATINIB,CHEMBL180022,3.05,4.47,12.55,8.81,5.93,8,2,557.05,557.05,2,112.40,40
3,2,CHEMBL475251,"{'molecule_chembl_id': 'CHEMBL475251', 'parent...","{'alogp': '3.63', 'aromatic_rings': 3, 'cx_log...",R-406,CHEMBL475251,3.63,3.63,10.90,3.05,3.63,10,3,470.46,470.46,0,128.75,34


In [8]:
###########################
#Filter the compounds based on their molecular properties (e.g. molecular weight < 400 amu), or max_phase, for example:
###########################

#Now keep only compounds with max_phase = 4 (ie approved drugs): 
res = mol_df[ mol_df['max_phase'] == 4 ]

#Display results:
res

,max_phase,molecule_chembl_id,molecule_hierarchy,molecule_properties,pref_name,parent_molecule_chembl_id,cx_logd,cx_logp,cx_most_apka,cx_most_bpka,alogp,hba,hbd,mw_freebase,full_mwt,num_ro5_violations,psa,heavy_atoms
0,4,CHEMBL535,"{'molecule_chembl_id': 'CHEMBL535', 'parent_ch...","{'alogp': '3.33', 'aromatic_rings': 2, 'cx_log...",SUNITINIB,CHEMBL535,1.28,2.93,11.46,9.04,3.33,3,3,398.48,398.48,0,77.23,29
2,4,CHEMBL180022,"{'molecule_chembl_id': 'CHEMBL180022', 'parent...","{'alogp': '5.93', 'aromatic_rings': 4, 'cx_log...",NERATINIB,CHEMBL180022,3.05,4.47,12.55,8.81,5.93,8,2,557.05,557.05,2,112.40,40


## Method 2: Use the python 'requests' module to access the API

* Pros: Use if don't want to install ChEMBL python client, e.g. if 'requests' module already in use as part of an existing workflow, or using a non-python coding language
* Cons: Slightly more code required than for ChEMBL python client. Need to include code to iterate over multiple pages of records.
* See https://docs.python-requests.org/en/latest/  

In [9]:
###############################
#This cell imports relevant python modules:
###############################
import pandas as pd #Use pandas python module to view and analyse data
import requests #This is used to access json files!

## 2a: Find compounds that are active on the specified targets:

In [10]:
###############################
#Search for activity for a list of targets:
###############################

#Specify the input parameters: 
targets = ['CHEMBL5552', 'CHEMBL1075195'] # Select a few example targets: Serine/threonine-protein kinase 25 (STK25) & Serine-threonine kinases TAO2.
targets = ",".join(targets) #Join the targets into a suitable string to fulfil the search conditions of the API
pchembl_value = 5 #Specify a minimum threshold of the pChEMBL activity value. Note that pCHEMBL = -log10(IC50, XC50, AC50, Ki, Kd, potency). Greater than or equal to 5 (10um) is a typical minimum rule of thumb for binding activity between a compound and a protein target. 
assay_type = 'B' #Only look for Binding Assays
cols = ",".join(['target_chembl_id', 'target_pref_name', 'parent_molecule_chembl_id','molecule_chembl_id','molecule_pref_name', 'pchembl_value']) # Only return the specified data fields 

###############################
url_stem = "https://www.ebi.ac.uk" #This is the stem of the url
url_string = url_stem + "/chembl/api/data/activity?target_chembl_id__in={}&pchembl_value__gte={}&assay_type={}&only={}&format=json".format(targets, pchembl_value, assay_type, cols) #This is the full url with the specified input parameters
url = requests.get( url_string ).json() #This calls the information back from the API using the 'requests' module, and converts it to json format
results = url['activities'] #This is a list of the results for activities

#This 'while' loop iterates over several pages of records (if required), and collates the list of results
#Remember that there is a limit to the number of records returned in any one API call (default is 20 records, maximum is 1000 records) e.g. include "&limit=50" in the url string
#So need to iterate over several pages of records to gather all relevant information together!
while url['page_meta']['next']:
    url = requests.get(url_stem + url['page_meta']['next']).json()
    results = results + url['activities'] #Add result (as a list) to previous list of results

#Convert the list of results into a Pandas dataframe:
act_df = pd.DataFrame(results)

#Print out some useful information:
print("This is the url string that calls the 'Activities' API with the initial query specification:\n{}".format(url_string) )
act_df

This is the url string that calls the 'Activities' API with the initial query specification:
https://www.ebi.ac.uk/chembl/api/data/activity?target_chembl_id__in=CHEMBL5552,CHEMBL1075195&pchembl_value__gte=5&assay_type=B&only=target_chembl_id,target_pref_name,parent_molecule_chembl_id,molecule_chembl_id,molecule_pref_name,pchembl_value&format=json


,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,target_chembl_id,target_pref_name,value
0,CHEMBL574738,AST-487,CHEMBL574738,5.92,CHEMBL5552,Serine/threonine-protein kinase 25,1200.0
1,CHEMBL522892,DOVITINIB,CHEMBL522892,5.92,CHEMBL5552,Serine/threonine-protein kinase 25,1200.0
2,CHEMBL607707,PELITINIB,CHEMBL607707,5.47,CHEMBL5552,Serine/threonine-protein kinase 25,3400.0
3,CHEMBL191003,JNJ-7706621,CHEMBL191003,6.44,CHEMBL5552,Serine/threonine-protein kinase 25,360.0
4,CHEMBL608533,MIDOSTAURIN,CHEMBL608533,5.75,CHEMBL5552,Serine/threonine-protein kinase 25,1800.0
...,...,...,...,...,...,...,...
78,CHEMBL388978,STAUROSPORINE,CHEMBL388978,8.57,CHEMBL5552,Serine/threonine-protein kinase 25,2.71
79,CHEMBL388978,STAUROSPORINE,CHEMBL388978,7.93,CHEMBL1075195,Serine/threonine-protein kinase TAO2,1.17
80,CHEMBL388978,STAUROSPORINE,CHEMBL388978,8.72,CHEMBL5552,Serine/threonine-protein kinase 25,1.9
81,CHEMBL4569508,None,CHEMBL4569508,5.57,CHEMBL1075195,Serine/threonine-protein kinase TAO2,2700.0


## 2b: Using the active compounds from the previous step, and call the 'molecule' API to find their molecular properties etc, so that the compound list can be prioritised

In [12]:
###############################
#First find the list of compounds that are within the act_df dataframe:
###############################
cmpd_chembl_ids = list(set(act_df['molecule_chembl_id']))
print("There are {} compounds initially identified as active on the known targets. e.g.{}...".format(len(cmpd_chembl_ids),cmpd_chembl_ids[0:2]))

There are 55 compounds initially identified as active on the known targets. e.g.['CHEMBL4568087', 'CHEMBL1721885']...


In [13]:
###############################
#For the identified compounds, extract their molecular properties and other information from the 'molecule' ChEMBL API
###############################

#Select a few examples of the active compounds from above (i.e. a reduced list!) : 
cmpd_chembl_ids = ",".join(['CHEMBL296468', 'CHEMBL180022', 'CHEMBL475251','CHEMBL535']) #Amend the format of the text string so that it is suitable for the API call
cols = ",".join([ 'molecule_chembl_id','pref_name', 'molecule_hierarchy', 'molecule_properties', 'max_phase']) # Only return the specified data fields 

###############################
url_stem = "https://www.ebi.ac.uk" #This is the stem of the url
url_string = url_stem + "/chembl/api/data/molecule?molecule_chembl_id__in={}&only={}&format=json".format(cmpd_chembl_ids,cols) #This is the full url with the specified input parameters
url = requests.get( url_string ).json() #This calls the information back from the API using the 'requests' module, and converts it to json format
results = url['molecules'] #This is a list of the results for activities

#This 'while' loop iterates over several pages of records (if required), and collates the list of results
#Remember that there is a limit to the number of records returned in any one API call (default is 20 records, maximum is 1000 records) e.g. include "&limit=50" in the url string
#So need to iterate over several pages of records to gather all relevant information together!
while url['page_meta']['next']:
    url = requests.get(url_stem + url['page_meta']['next']).json()
    results = results + url['molecules'] #Add result (as a list) to previous list of results

#Convert the list of results into a Pandas dataframe:
mol_df = pd.DataFrame(results)

#Print out some useful information:
print("This is the url string that calls the 'Molecule' API with the specified query\n{}".format(url_string) )
mol_df

This is the url string that calls the 'Molecule' API with the specified query
https://www.ebi.ac.uk/chembl/api/data/molecule?molecule_chembl_id__in=CHEMBL296468,CHEMBL180022,CHEMBL475251,CHEMBL535&only=molecule_chembl_id,pref_name,molecule_hierarchy,molecule_properties,max_phase&format=json


,max_phase,molecule_chembl_id,molecule_hierarchy,molecule_properties,pref_name
0,4,CHEMBL535,"{'molecule_chembl_id': 'CHEMBL535', 'parent_ch...","{'alogp': '3.33', 'aromatic_rings': 2, 'cx_log...",SUNITINIB
1,1,CHEMBL296468,"{'molecule_chembl_id': 'CHEMBL296468', 'parent...","{'alogp': '3.66', 'aromatic_rings': 2, 'cx_log...",BMS-387032
2,4,CHEMBL180022,"{'molecule_chembl_id': 'CHEMBL180022', 'parent...","{'alogp': '5.93', 'aromatic_rings': 4, 'cx_log...",NERATINIB
3,2,CHEMBL475251,"{'molecule_chembl_id': 'CHEMBL475251', 'parent...","{'alogp': '3.63', 'aromatic_rings': 3, 'cx_log...",R-406


In [14]:
###########################
# Convert nested cells (ie those containing a dictionary) to individual columns in the dataframe so that is it easier to filter!
###########################
# Molecule hierarchy: 
mol_df['parent_chembl_id'] = mol_df['molecule_hierarchy'].apply(lambda x: x['parent_chembl_id'])

#Physicochemical properties (only report if cells are not null)
mol_df['cx_logd'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_logd'])
mol_df['cx_logp'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_logp'])
mol_df['cx_most_apka'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_most_apka'])
mol_df['cx_most_bpka'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['cx_most_bpka'])
mol_df['alogp'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['alogp'])
mol_df['hba'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['hba'])
mol_df['hbd'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['hbd'])
mol_df['mw_freebase'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['mw_freebase']) #This is the mwt of the parent compound
mol_df['full_mwt'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['full_mwt']) #This is the mwt of the full compound including any salt
mol_df['num_ro5_violations'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['num_ro5_violations'])
mol_df['psa'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['psa'])
mol_df['heavy_atoms'] = mol_df.loc[ mol_df['molecule_properties'].notnull(), 'molecule_properties'].apply(lambda x: x['heavy_atoms'])

mol_df

(4, 18)


,max_phase,molecule_chembl_id,molecule_hierarchy,molecule_properties,pref_name,parent_chembl_id,cx_logd,cx_logp,cx_most_apka,cx_most_bpka,alogp,hba,hbd,mw_freebase,full_mwt,num_ro5_violations,psa,heavy_atoms
0,4,CHEMBL535,"{'molecule_chembl_id': 'CHEMBL535', 'parent_ch...","{'alogp': '3.33', 'aromatic_rings': 2, 'cx_log...",SUNITINIB,CHEMBL535,1.28,2.93,11.46,9.04,3.33,3,3,398.48,398.48,0,77.23,29
1,1,CHEMBL296468,"{'molecule_chembl_id': 'CHEMBL296468', 'parent...","{'alogp': '3.66', 'aromatic_rings': 2, 'cx_log...",BMS-387032,CHEMBL296468,0.33,0.94,7.94,10.17,3.66,7,2,380.54,380.54,0,80.05,25
2,4,CHEMBL180022,"{'molecule_chembl_id': 'CHEMBL180022', 'parent...","{'alogp': '5.93', 'aromatic_rings': 4, 'cx_log...",NERATINIB,CHEMBL180022,3.05,4.47,12.55,8.81,5.93,8,2,557.05,557.05,2,112.40,40
3,2,CHEMBL475251,"{'molecule_chembl_id': 'CHEMBL475251', 'parent...","{'alogp': '3.63', 'aromatic_rings': 3, 'cx_log...",R-406,CHEMBL475251,3.63,3.63,10.90,3.05,3.63,10,3,470.46,470.46,0,128.75,34


In [16]:
###########################
#Filter the compounds based on their molecular properties (e.g. molecular weight < 400 amu), or max_phase, for example:
###########################

#Now keep only compounds with max_phase = 4 (ie approved drugs), for example: 
res = mol_df[ mol_df['max_phase'] == 4 ]

#Display results:
res

,max_phase,molecule_chembl_id,molecule_hierarchy,molecule_properties,pref_name,parent_chembl_id,cx_logd,cx_logp,cx_most_apka,cx_most_bpka,alogp,hba,hbd,mw_freebase,full_mwt,num_ro5_violations,psa,heavy_atoms
0,4,CHEMBL535,"{'molecule_chembl_id': 'CHEMBL535', 'parent_ch...","{'alogp': '3.33', 'aromatic_rings': 2, 'cx_log...",SUNITINIB,CHEMBL535,1.28,2.93,11.46,9.04,3.33,3,3,398.48,398.48,0,77.23,29
2,4,CHEMBL180022,"{'molecule_chembl_id': 'CHEMBL180022', 'parent...","{'alogp': '5.93', 'aromatic_rings': 4, 'cx_log...",NERATINIB,CHEMBL180022,3.05,4.47,12.55,8.81,5.93,8,2,557.05,557.05,2,112.40,40
